In [42]:
import sqlite3

# Create or open the database file
conn = sqlite3.connect("data/imdb.db")


cur = conn.cursor()

print("Database created successfully at data/imdb.db")


Database created successfully at data/imdb.db


In [43]:
import pandas as pd

# Read the first file
df_basics = pd.read_csv("raw_imdb/title.basics.tsv", sep="\t", na_values="\\N")

# Keep only important columns
df_basics = df_basics[[
    "tconst", "titleType", "primaryTitle", "originalTitle",
    "isAdult", "startYear", "endYear", "runtimeMinutes", "genres"
]]

# Store it inside the database as a table
df_basics.to_sql("title_basics", conn, if_exists="append", index=False)

print("title_basics table added to imdb.db")


/var/folders/tb/lmlh_5fd45s7sv05vl8p6nb80000gn/T/ipykernel_66206/936323268.py:4: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



title_basics table added to imdb.db


In [44]:
pd.read_sql_query("SELECT COUNT(*) FROM title_basics;", conn)


,COUNT(*)
0,24039656


In [45]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,title_basics
1,title_ratings
2,name_basics
3,title_principals


In [46]:
df_ratings = pd.read_csv("raw_imdb/title.ratings.tsv", sep="\t", na_values="\\N")
df_ratings.to_sql("title_ratings", conn, if_exists="replace", index=False)
print("title_ratings table added to imdb.db")


title_ratings table added to imdb.db


In [47]:
pd.read_sql_query("SELECT COUNT(*) FROM title_ratings;", conn)


,COUNT(*)
0,1631685


In [48]:
#Check if the tables are there and has a common id
query = """
SELECT b.primaryTitle, b.startYear, r.averageRating, r.numVotes
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
LIMIT 5;
"""
pd.read_sql_query(query, conn)


,primaryTitle,startYear,averageRating,numVotes
0,Carmencita,1894.0,5.7,2182
1,Le clown et ses chiens,1892.0,5.5,302
2,Poor Pierrot,1892.0,6.4,2262
3,Un bon bock,1892.0,5.2,194
4,Blacksmith Scene,1893.0,6.2,2999


In [49]:
# Load the 'name.basics.tsv' file



df_names = pd.read_csv("raw_imdb/name.basics.tsv", sep="\t", na_values="\\N")

# Keep only key columns to make it smaller and faster.
df_names = df_names[[
    "nconst",          # unique person ID
    "primaryName",     # person's main name
    "birthYear",       # birth year
    "deathYear",       # death year (optional)
    "primaryProfession", # e.g., actor, director
    "knownForTitles"   # comma-separated list of title IDs they are known for
]]


df_names.to_sql("name_basics", conn, if_exists="append", index=False)
print("name_basics table added to imdb.db")


#  Load the 'title.principals.tsv' file


df_principals = pd.read_csv("raw_imdb/title.principals.tsv", sep="\t", na_values="\\N")

# Keep only important columns for relationships.
df_principals = df_principals[[
    "tconst",          # title ID (links to title_basics)
    "ordering",        # position of the person in credits
    "nconst",          # person ID (links to name_basics)
    "category",        # job type (actor, director, writer, etc.)
    "job",             # optional description
    "characters"       # roles played (if any)
]]


df_principals.to_sql("title_principals", conn, if_exists="append", index=False)
print("title_principals table added to imdb.db")


name_basics table added to imdb.db
title_principals table added to imdb.db


In [50]:
pd.read_sql_query("SELECT COUNT(*) FROM title_principals;", conn)


,COUNT(*)
0,191116066


In [51]:
pd.read_sql_query("SELECT COUNT(*) FROM name_basics;", conn)


,COUNT(*)
0,29674096


In [52]:
#  Verify all tables exist 
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,title_basics
1,name_basics
2,title_principals
3,title_ratings


In [53]:
#  Get row counts for all tables 

tables = ["title_basics", "title_ratings", "name_basics", "title_principals"]

for t in tables:
    result = pd.read_sql_query(f"SELECT COUNT(*) AS count FROM {t};", conn)
    print(f"{t}: {result['count'][0]:,} rows")


title_basics: 24,039,656 rows
title_ratings: 1,631,685 rows
name_basics: 29,674,096 rows
title_principals: 191,116,066 rows


In [54]:
#  Verify title_basics ↔ title_ratings link 
# Both tables share 'tconst' as the common key.


query = """
SELECT b.primaryTitle AS Title,
       b.startYear AS Year,
       b.genres AS Genre,
       r.averageRating AS Rating,
       r.numVotes AS Votes
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
ORDER BY r.numVotes DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)


,Title,Year,Genre,Rating,Votes
0,The Shawshank Redemption,1994.0,Drama,9.3,3114529
1,The Shawshank Redemption,1994.0,Drama,9.3,3114529
2,The Dark Knight,2008.0,"Action,Crime,Drama",9.1,3090349
3,The Dark Knight,2008.0,"Action,Crime,Drama",9.1,3090349
4,Inception,2010.0,"Action,Adventure,Sci-Fi",8.8,2745199
5,Inception,2010.0,"Action,Adventure,Sci-Fi",8.8,2745199
6,Fight Club,1999.0,"Crime,Drama,Thriller",8.8,2529683
7,Fight Club,1999.0,"Crime,Drama,Thriller",8.8,2529683
8,Game of Thrones,2011.0,"Action,Adventure,Drama",9.2,2495580
9,Game of Thrones,2011.0,"Action,Adventure,Drama",9.2,2495580


In [22]:
#  Verify title_basics ↔ title_ratings ↔ title_principals ↔ name_basics 
# This join ensures movie, rating, and people data all link correctly.

query = """
SELECT b.primaryTitle AS Movie,
       b.startYear AS Year,
       r.averageRating AS Rating,
       n.primaryName AS Director
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
JOIN title_principals p ON b.tconst = p.tconst
JOIN name_basics n ON p.nconst = n.nconst
WHERE p.category = 'director'
ORDER BY r.averageRating DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)


,Movie,Year,Rating,Director
0,Renegades 2,1995.0,10.0,Buck Adams
1,God Comes as a Child,1998.0,10.0,Jeremiah Hayes
2,Innocent Bi-Standers,1989.0,10.0,Ron Jeremy
3,Closed for Business,1997.0,10.0,James H. Brown
4,Heroes,2000.0,10.0,Eduardo Moral
5,Lana & Theresa Having It Off,1999.0,10.0,Mark Matthews
6,The Terrapin,2002.0,10.0,Regis Trigano
7,Heroes,2002.0,10.0,Jackie Chenoweth
8,The Outlands,2002.0,10.0,Jon Simpson
9,Doli: The Fragments of My Chilhood,1992.0,10.0,Zrinko Ogresta


In [55]:
#  Top-rated movies by popular actors 
# Same as before, but filters on category='actor'.

query = """
SELECT b.primaryTitle AS Movie,
       b.startYear AS Year,
       r.averageRating AS Rating,
       n.primaryName AS Actor
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
JOIN title_principals p ON b.tconst = p.tconst
JOIN name_basics n ON p.nconst = n.nconst
WHERE p.category = 'actor'
ORDER BY r.averageRating DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)


,Movie,Year,Rating,Actor
0,Renegades 2,1995.0,10.0,Buck Adams
1,Renegades 2,1995.0,10.0,Buck Adams
2,Renegades 2,1995.0,10.0,Buck Adams
3,Renegades 2,1995.0,10.0,Buck Adams
4,Renegades 2,1995.0,10.0,T.T. Boy
5,Renegades 2,1995.0,10.0,T.T. Boy
6,Renegades 2,1995.0,10.0,T.T. Boy
7,Renegades 2,1995.0,10.0,T.T. Boy
8,Renegades 2,1995.0,10.0,J.W. Gaay
9,Renegades 2,1995.0,10.0,J.W. Gaay


In [56]:
#Visualization with Plotly 

!pip install plotly



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [31]:

import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [33]:
#  Average IMDb Rating by Genre


query = """
SELECT b.genres AS Genre,
       ROUND(AVG(r.averageRating), 2) AS Avg_Rating,
       COUNT(*) AS Movie_Count
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
WHERE b.genres IS NOT NULL AND b.genres NOT LIKE '%Adult%'
GROUP BY b.genres
HAVING Movie_Count > 1000
ORDER BY Avg_Rating DESC
LIMIT 15;
"""


df_genre = pd.read_sql_query(query, conn)


import plotly.express as px

fig = px.bar(df_genre,
             x="Genre",
             y="Avg_Rating",
             text="Avg_Rating",
             color="Avg_Rating",
             color_continuous_scale="Viridis",
             title=" Average IMDb Rating by Genre (Top 15 Genres)")

fig.update_traces(textposition="outside")
fig.update_layout(xaxis_title="Genre", yaxis_title="Average Rating")
fig.show()


In [34]:
# Movie Release Trend by Decade ---

# SQL query → Count how many movies were released each decade.
query = """
SELECT (b.startYear/10)*10 AS Decade,
       COUNT(*) AS Movie_Count
FROM title_basics b
WHERE b.titleType = 'movie'
  AND b.startYear BETWEEN 1900 AND 2025
GROUP BY Decade
ORDER BY Decade;
"""


df_decade = pd.read_sql_query(query, conn)


import plotly.express as px
fig = px.line(df_decade,
              x="Decade",
              y="Movie_Count",
              markers=True,
              title=" Number of Movies Released per Decade")


fig.update_layout(xaxis_title="Decade", yaxis_title="Number of Movies Released")
fig.show()


In [38]:
# Top Directors by Average IMDb Rating

# SQL query → find directors with the highest average ratings.
#  include directors who have directed at least 3 rated movies.

query = """
SELECT n.primaryName AS Director,
       ROUND(AVG(r.averageRating), 2) AS Avg_Rating,
       COUNT(DISTINCT b.tconst) AS Movie_Count
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
JOIN title_principals p ON b.tconst = p.tconst
JOIN name_basics n ON p.nconst = n.nconst
WHERE p.category = 'director'
GROUP BY n.primaryName
HAVING Movie_Count >= 3
ORDER BY Avg_Rating DESC
LIMIT 10;
"""


df_directors = pd.read_sql_query(query, conn)

import plotly.express as px
fig = px.bar(df_directors,
             x="Director",
             y="Avg_Rating",
             text="Avg_Rating",
             color="Avg_Rating",
             color_continuous_scale="Cividis",
             title=" Top 10 Directors by Average IMDb Rating (Min. 3 Movies)")


fig.update_traces(textposition="outside")
fig.update_layout(xaxis_title="Director", yaxis_title="Average IMDb Rating")
fig.show()

In [39]:
# Top 15 Most-Voted Movies ---

query = """
SELECT b.primaryTitle AS Movie,
       b.startYear AS Year,
       r.averageRating AS Rating,
       r.numVotes AS Votes
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
WHERE b.titleType = 'movie'
  AND r.numVotes > 100000     -- ensure popular films
ORDER BY r.numVotes DESC
LIMIT 15;
"""

df_popular = pd.read_sql_query(query, conn)

import plotly.express as px
fig = px.bar(df_popular,
             x="Movie",
             y="Rating",
             color="Votes",
             text="Rating",
             color_continuous_scale="Plasma",
             title=" Top 15 Most-Voted Movies on IMDb")

fig.update_traces(textposition="outside")
fig.update_layout(xaxis_title="Movie", yaxis_title="IMDb Rating")
fig.show()


In [40]:
# Genre vs. Average Rating (Bubble chart) ---

query = """
SELECT b.genres AS Genre,
       ROUND(AVG(r.averageRating), 2) AS Avg_Rating,
       COUNT(*) AS Movie_Count
FROM title_basics b
JOIN title_ratings r ON b.tconst = r.tconst
WHERE b.genres IS NOT NULL AND b.genres NOT LIKE '%Adult%'
GROUP BY b.genres
HAVING Movie_Count > 1000
ORDER BY Avg_Rating DESC
LIMIT 20;
"""

df_genre_bubble = pd.read_sql_query(query, conn)

fig = px.scatter(df_genre_bubble,
                 x="Movie_Count",
                 y="Avg_Rating",
                 size="Movie_Count",
                 color="Genre",
                 hover_name="Genre",
                 title=" IMDb Genre Ratings vs. Number of Movies",
                 size_max=60)

fig.update_layout(xaxis_title="Number of Movies", yaxis_title="Average Rating")
fig.show()


In [58]:
from sqlalchemy import create_engine, inspect

engine = create_engine("sqlite:///data/imdb.db")  # <-- adjust path if needed
inspector = inspect(engine)

tables = inspector.get_table_names()
print("📋 Tables found in IMDb database:")
for t in tables:
    print("-", t)

📋 Tables found in IMDb database:
- name_basics
- title_basics
- title_principals
- title_ratings


In [59]:
import os

# List everything in your current working directory
print("📁 Current working directory:", os.getcwd())
print("\n📂 Files & folders:")
for f in os.listdir():
    print("-", f)


📁 Current working directory: /Users/aditi/Documents/GenQuery

📂 Files & folders:
- raw_imdb
- imdb.db
- .DS_Store
- App
- 02_ai_query_engine.ipynb
- .env
- rag_imdb
- .ipynb_checkpoints
- data
- 01_build_imdb_db.ipynb


In [60]:
import os
db_path = "data/imdb.db"
print("Exists:", os.path.exists(db_path))
print("File size (MB):", round(os.path.getsize(db_path)/1e6, 2))


Exists: True
File size (MB): 13732.78


In [61]:
import os
db_path = "data/imdb.db"
print("Exists:", os.path.exists(db_path))
print("File size (MB):", round(os.path.getsize(db_path)/1e6, 2))


Exists: True
File size (MB): 13732.78
